<a href="https://colab.research.google.com/github/shonnvs-code/Factor-Based-Fund-Analysis-Portfolio-Modeling/blob/main/Model/ETF_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from tabulate import tabulate
from google.colab import drive
drive.mount('/content/drive')
etfs = pd.read_csv('/content/drive/My Drive/Team 11_BA815_Competing with Analytics/Data/ETFs.csv')
etfs.info()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2310 entries, 0 to 2309
Columns: 142 entries, fund_symbol to fund_treynor_ratio_10years
dtypes: float64(124), int64(1), object(17)
memory usage: 2.5+ MB


In [ ]:
# ============================================================
# 2. KEEP ONLY NECESSARY ETF COLUMNS
# ============================================================
numeric_features_fund = [
    # returns
    "fund_return_1year", "fund_return_3years", "fund_return_5years", "fund_return_10years",
    # risk/quality
    "fund_alpha_3years", "fund_alpha_5years",
    "fund_beta_3years", "fund_beta_5years",
    "fund_stdev_3years", "fund_stdev_5years",
    "fund_sharpe_ratio_3years", "fund_sharpe_ratio_5years",
    # costs
    "fund_annual_report_net_expense_ratio",
    "annual_holdings_turnover",
    # liquidity/size
    "total_net_assets",
    "avg_vol_3month"
]

categorical_features_fund = [
    "fund_category", "region", "fund_family",
    "investment_type", "size_type"
]

id_display_cols = ["fund_symbol", "fund_short_name", "fund_long_name"]

# keep only columns that exist (safe)
keep_cols = [c for c in numeric_features_fund + categorical_features_fund + id_display_cols if c in etfs.columns]
etfs = etfs[keep_cols].copy()

# basic NA handling for key cost fields
for c in ["fund_annual_report_net_expense_ratio", "annual_holdings_turnover"]:
    if c in etfs.columns:
        etfs[c] = etfs.groupby("fund_category")[c].transform(lambda x: x.fillna(x.median()))
        etfs[c] = etfs[c].fillna(etfs[c].median())

# fill remaining numeric NaNs with column median (quick cleanup)
for c in numeric_features_fund:
    if c in etfs.columns:
        etfs[c] = etfs[c].fillna(etfs[c].median())

# fill remaining categorical NaNs with mode
for c in categorical_features_fund:
    if c in etfs.columns:
        etfs[c] = etfs[c].fillna(etfs[c].mode(dropna=True)[0])



In [ ]:
# ============================================================
# 3. GENERATE SYNTHETIC INVESTORS (FOR TRAINING)
# ============================================================
def generate_investors(n=800, seed=42):
    rng = np.random.default_rng(seed)
    investors = pd.DataFrame({
        "risk_tolerance": rng.choice(["low", "medium", "high"], size=n, p=[0.35, 0.45, 0.20]),
        "investment_horizon_years": rng.choice([3, 5, 10], size=n, p=[0.20, 0.30, 0.30, 0.20]),
        "income_level": rng.choice(["low", "medium", "high"], size=n, p=[0.30, 0.50, 0.20]),
        "active_preference": rng.choice(["active", "passive", "either"], size=n, p=[0.30, 0.50, 0.20]),
    })
    investors["investor_id"] = np.arange(n)
    return investors

investors = generate_investors()



In [ ]:
# ============================================================
# 4. BUILD ALL (INVESTOR × ETF) PAIRS
# ============================================================
pairs = investors.merge(etfs, how="cross")



In [ ]:
# ============================================================
# 5. DEFINE SUPERVISED TARGET = UTILITY SCORE
# ============================================================
def percentile_rank(s):
    return s.rank(pct=True)

# --- fund scores normalized 0–1 ---
pairs["ret_score"] = percentile_rank(
    pairs["fund_return_5years"].fillna(pairs["fund_return_3years"])
)
pairs["alpha_score"] = percentile_rank(
    pairs["fund_alpha_5years"].fillna(pairs["fund_alpha_3years"])
)
pairs["sharpe_score"] = percentile_rank(
    pairs["fund_sharpe_ratio_5years"].fillna(pairs["fund_sharpe_ratio_3years"])
)

vol_proxy = pairs["fund_stdev_5years"].fillna(pairs["fund_stdev_3years"])
pairs["vol_score"] = 1 - percentile_rank(vol_proxy)

pairs["fee_score"] = 1 - percentile_rank(pairs["fund_annual_report_net_expense_ratio"])
pairs["turnover_score"] = 1 - percentile_rank(pairs["annual_holdings_turnover"])
pairs["liq_score"] = percentile_rank(pairs["total_net_assets"].fillna(0))

# --- investor penalties/bonuses ---
# Risk mismatch
risk_map = {"low": 0.3, "medium": 0.6, "high": 1.0}
pairs["risk_tol_factor"] = pairs["risk_tolerance"].map(risk_map)
pairs["product_risk"] = percentile_rank(vol_proxy)
pairs["risk_mismatch"] = np.maximum(0, pairs["product_risk"] - pairs["risk_tol_factor"])

# Income effects:
# - higher income cares more about liquidity/scalability
# - lower income is more fee sensitive
income_liq_map = {"low": 0.6, "medium": 1.0, "high": 1.4}
income_fee_map = {"low": 1.4, "medium": 1.0, "high": 0.7}
pairs["liq_weight"] = pairs["income_level"].map(income_liq_map)
pairs["fee_weight"] = pairs["income_level"].map(income_fee_map)

# Active/passive preference bonus
def active_bonus(row):
    # crude active flag from investment_type/strategy text
    strategy_text = (str(row.get("investment_type","")) + " " +
                     str(row.get("investment_strategy",""))).lower()
    fund_is_active = int("active" in strategy_text)

    if row["active_preference"] == "active":
        return fund_is_active
    if row["active_preference"] == "passive":
        return 1 - fund_is_active
    return 0.5  # either

pairs["active_match_bonus"] = pairs.apply(active_bonus, axis=1)

# --- FINAL UTILITY ---
pairs["utility"] = (
    2.0 * pairs["ret_score"]
  + 1.0 * pairs["alpha_score"]
  + 1.5 * pairs["sharpe_score"]
  + pairs["liq_weight"] * pairs["liq_score"]
  + 0.8 * pairs["active_match_bonus"]
  - 2.5 * pairs["risk_mismatch"]
  - pairs["fee_weight"] * (1 - pairs["fee_score"])
  - 1.0 * (1 - pairs["turnover_score"])
)



In [ ]:

# ============================================================
# 6. TRAIN GLOBAL UTILITY REGRESSOR
# ============================================================
target = "utility"

numeric_features = ["investment_horizon_years"] + numeric_features_fund
numeric_features = [c for c in numeric_features if c in pairs.columns]

categorical_features = [
    "risk_tolerance", "income_level", "active_preference"
] + categorical_features_fund
categorical_features = [c for c in categorical_features if c in pairs.columns]

X = pairs[numeric_features + categorical_features]
y = pairs[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

model = GradientBoostingRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=3,
    random_state=42
)

utility_regressor = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", model)
])

utility_regressor.fit(X_train, y_train)

pred = utility_regressor.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, pred))
print("Utility regression RMSE:", rmse)


Utility regression RMSE: 0.10923877969219653


In [ ]:

# ============================================================
# 7. USER INPUT → TOP 5 ETF RECOMMENDATIONS
# ============================================================
def get_user_profile():
    risk = input("Risk tolerance (low/medium/high): ").strip().lower()
    while risk not in ["low", "medium", "high"]:
        risk = input("Enter low, medium, or high: ").strip().lower()

    horizon = input("Investment horizon in years (e.g., 1,5,10,20): ").strip()
    while not horizon.isdigit() or int(horizon) <= 0:
        horizon = input("Enter a positive integer: ").strip()
    horizon = int(horizon)

    income = input("Income level (low/medium/high): ").strip().lower()
    while income not in ["low", "medium", "high"]:
        income = input("Enter low, medium, or high: ").strip().lower()

    active_pref = input("Active preference (active/passive/either): ").strip().lower()
    while active_pref not in ["active", "passive", "either"]:
        active_pref = input("Enter active, passive, or either: ").strip().lower()

    return {
        "risk_tolerance": risk,
        "horizon": horizon,
        "income_level": income,
        "active_preference": active_pref
    }

def recommend_top_etfs(user_profile, etf_df, model, top_k=5):
    inv = pd.DataFrame([user_profile]).copy()
    inv["investment_horizon_years"] = inv.pop("horizon")
    inv["investor_id"] = -1

    rec_pairs = inv.merge(etf_df, how="cross")
    rec_X = rec_pairs[numeric_features + categorical_features]

    rec_pairs["pred_utility"] = model.predict(rec_X)

    # Fixed mapping from approx [-6, +6] to [0,100]
    rec_pairs["score_0_100"] = ((rec_pairs["pred_utility"] + 6) / 12) * 100
    rec_pairs["score_0_100"] = rec_pairs["score_0_100"].clip(0, 100)

    out = rec_pairs.sort_values("score_0_100", ascending=False)

    # prefer full long name, fall back to short name
    out["fund_name"] = out["fund_long_name"].fillna(out["fund_short_name"])

    return out[[
        "fund_name", "fund_category",
        "fund_annual_report_net_expense_ratio",
        "score_0_100"
    ]].head(top_k)

def describe_investor(inv):
    print("\nRecommendations generated for:")
    print(f"- Risk tolerance: {inv['risk_tolerance']}")
    print(f"- Investment horizon: {inv['horizon']} years")
    print(f"- Income level: {inv['income_level']}")
    print(f"- Active preference: {inv['active_preference']}\n")

# ---- Run the user-facing recommender ----
user_profile = get_user_profile()
describe_investor(user_profile)

top5 = recommend_top_etfs(user_profile, etfs, utility_regressor, top_k=5)

# Pretty display
clean = top5.rename(columns={
    "fund_name": "ETF Name",
    "fund_category": "Category",
    "fund_annual_report_net_expense_ratio": "Expense Ratio",
    "score_0_100": "Match Score (0-100)"
})

clean["Expense Ratio"] = clean["Expense Ratio"].map(lambda x: f"{x:.3%}" if pd.notna(x) else "NA")
clean["Match Score (0-100)"] = clean["Match Score (0-100)"].round(1)

print(tabulate(clean, headers="keys", tablefmt="fancy_grid", showindex=False))